In [70]:
import sys,os
sys.path.insert(0, "D:/Dropbox/Progetti/SynthBurst")

In [71]:
import pandas as pd
import numpy as np
float_formatter = "{:.2f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
sns.set_theme(style="white")
sns.set_context("notebook")

In [72]:
seeding = True

if seeding:
    seed_value = 66
    np.random.seed(seed_value)

In [73]:
import synthburst.simulate as sb

# Telemetry Budget Simulation
# Alternative telemetry model: ratemeters dominated

<i>This alternative scenario takes inspiration from the original telemetry model employed by Fermi-GBM. In the early phases of its operations, Fermi did not download all the data products of the GBM experiment. Only two different binned datasets were downloaded throughout the day (CSPEC and CTIME data). Photon by photon data products (TTE) were downloaded only when on-board trigger resolved. Fermi later changed this model to one in which all data products were downloaded in full resolution.</i>

We simulate the photon lists for:
2. one full-band, bright burst with duration $\sim 50$ s and typical Band emission ($\alpha=-1$, $\beta=-2.33$, $E_p=174$).
3. one full-band, extremely bright burst with duration $\sim 50$ s and typical Band emission (as above).

Starting from these data we estimate:
1. the number of headers for each simulation, assuming that it is possible to accomodate a number of events `BUFFER_LEN` (nominally $8192$) before LAM, that each photons detected by scintillators results in two events and a new buffer file is created with period equal `BUFFER_REFRESH_S` (nominally $1$ s). 
2. the number of photons detected by SDDs (`X`).
3. the number of photons detected by scintillators (`S`).

Finally we compute the actual telemetry budget assuming that:
1. each header weights `HEADER_BITSIZE` bits (nominally set to 928).
2. each `X` photon results in a single record event with dimension `SINGLE_EVENT_BITSIZE` (nominally 64 bit).
3. each `S` photon results in a double record event with dimension `DOUBLE_EVENT_BITSIZE` (nominally 96 bit).
This implies that event with multiplicity $> 2$ will be dealt with as event with multiplicity $= 2$.
4. telemetry of scientific ratemeters.


### Data Format Specifications

In [74]:
SINGLE_EVENT_BITSIZE = 32 + 32 # TIME EVENT + PIXEL EVENT
DOUBLE_EVENT_BITSIZE = 32 + 2*32 # TIME EVENT + 2 PIXEL EVENT

HEADER_BITSIZE = 928
BUFFER_LEN = 8192
BUFFER_REFRESH_S = 1.0

### Constants

In [75]:
SECONDS_IN_A_DAY = 24*60*60
QUADRANTS_NUM = 4

-----

## Scientific ratemeters

In this model use two different scientific rate meters data products. 
The first one is obtained binning data in 8 different bands, separately for 4 quadrants, with binning 0.0625 s. A GPS header is added only once a second.  
The second SR data product is obtained binning the data in 64 energy channels, separately for all quadrants, with binning 4.0 s.
We assume 8 bit counts and 4 bytes GPS headers.

The expense for the time-resolved ratemeters:

In [76]:
TRSR_REFRESH_PERIOD = 0.256 # seconds
TRSR_COUNTS_BITSIZE = 8
TRSR_GPS_BITSIZE = 4*8
TRSR_BANDS = 8

TRSR_TELEMETRY_BITS = (QUADRANTS_NUM*TRSR_BANDS*TRSR_COUNTS_BITSIZE/TRSR_REFRESH_PERIOD + TRSR_GPS_BITSIZE)*(SECONDS_IN_A_DAY)
TRSR_TELEMETRY_GBITS = TRSR_TELEMETRY_BITS/10**9
print("the TRSR daily telemetry is {} Gbit.".format(TRSR_TELEMETRY_GBITS))

the TRSR daily telemetry is 0.0891648 Gbit.


The expense for the energy-resolved ratemeters:

In [77]:
ERSR_REFRESH_PERIOD = 4.096 # seconds
ERSR_COUNTS_BITSIZE = 8
ERSR_GPS_BITSIZE = 4*8
ERSR_BANDS = 64

ERSR_TELEMETRY_BITS = (QUADRANTS_NUM*ERSR_BANDS*ERSR_COUNTS_BITSIZE/ERSR_REFRESH_PERIOD + ERSR_GPS_BITSIZE/ERSR_REFRESH_PERIOD)*(SECONDS_IN_A_DAY)
ERSR_TELEMETRY_GBITS = ERSR_TELEMETRY_BITS/10**9
print("the ERSR daily telemetry is {} Gbit.".format(ERSR_TELEMETRY_GBITS))

the ERSR daily telemetry is 0.043875 Gbit.


In [78]:
SR_TELEMETRY_GBITS = ERSR_TELEMETRY_GBITS + TRSR_TELEMETRY_GBITS
print("the SR daily telemetry is {} Gbit.".format(SR_TELEMETRY_GBITS))

the SR daily telemetry is 0.13303979999999999 Gbit.


## Burst telemetry

We use the data obtained with the baseline simulation. Since minimal headers have small impact on burst telemetry we neglect them.

### Common burst telemetry

In [66]:
COMMONBURST_TELEMETRY_GBITS = 0.03041

### Very bright burst

In [65]:
BRIGHTBURST_TELEMETRY_GBITS = 0.08622

---

# Conclusions

The estimated total daily budget is :

In [62]:
TOTAL_BUDGET_GBITS = (BRIGHTBURST_TELEMETRY_GBITS + 
                      COMMONBURST_TELEMETRY_GBITS + 
                      SR_TELEMETRY_GBITS)

print("the total daily budget is {:.2f} Gbit.".format(TOTAL_BUDGET_GBITS))

the total daily budget is 0.25 Gbit.


Note that this calculation does not include any margin. On the brightside however, the impact of LAMs on the telemetry is taken into account, at least to some degree. Indeed, the present estimate of LAM headers is an underestimate due to the assumptions that the maximum event multiplicity equals 2. This situation can be improved providing an estimate of the incidence of high multiplicity events relative to single/double events.

Considering a 30% margin we obtain:

In [58]:
MARGIN = 30
MARGIN_BUDGET_GBITS = (BRIGHTBURST_TELEMETRY_GBITS + 
                      COMMONBURST_TELEMETRY_GBITS)*MARGIN/100

print("the total daily budget with {}% margin is {:.2f} Gbit.".format(MARGIN, TOTAL_BUDGET_GBITS + MARGIN_BUDGET_GBITS))

the total daily budget with 30% margin is 0.28 Gbit.


For a 95% margin:

In [59]:
MARGIN = 95
MARGIN_BUDGET_GBITS = (BRIGHTBURST_TELEMETRY_GBITS + 
                      COMMONBURST_TELEMETRY_GBITS)*MARGIN/100

print("the total daily budget with {}% margin is {:.2f} Gbit.".format(MARGIN, TOTAL_BUDGET_GBITS + MARGIN_BUDGET_GBITS))

the total daily budget with 95% margin is 0.36 Gbit.
